In [12]:
# Crawl


import requests
from bs4 import BeautifulSoup

url = "https://fullstackopen.com"

prefix_urls = []
for i in range(14):
    prefix_urls.append(url+"/en/part"+str(i))
prefix_urls

urls = []
for prefix_url in prefix_urls:
    reqs = requests.get(prefix_url)
    #encoding = "utf-8"
    soup = BeautifulSoup(reqs.content, 'html.parser')

    for link in soup.find_all('a'):
        link = link.get('href')
        if "part" in link and link not in urls:
            urls.append(link)

for j in range(14):
    urls.remove("/en/part"+str(j))

urls

['/en/part0/general_info',
 '/en/part0/fundamentals_of_web_apps',
 '/en/part1/introduction_to_react',
 '/en/part1/java_script',
 '/en/part1/component_state_event_handlers',
 '/en/part1/a_more_complex_state_debugging_react_apps',
 '/en/part2/rendering_a_collection_modules',
 '/en/part2/forms',
 '/en/part2/getting_data_from_server',
 '/en/part2/altering_data_in_server',
 '/en/part2/adding_styles_to_react_app',
 '/en/part3/node_js_and_express',
 '/en/part3/deploying_app_to_internet',
 '/en/part3/saving_data_to_mongo_db',
 '/en/part3/validation_and_es_lint',
 '/en/part4/structure_of_backend_application_introduction_to_testing',
 '/en/part4/testing_the_backend',
 '/en/part4/user_administration',
 '/en/part4/token_authentication',
 '/en/part4/legacy_testing_with_jest',
 '/en/part5/login_in_frontend',
 '/en/part5/props_children_and_proptypes',
 '/en/part5/testing_react_apps',
 '/en/part5/end_to_end_testing_playwright',
 '/en/part5/end_to_end_testing_cypress',
 '/en/part6/flux_architecture_and

In [14]:
len(urls)

60

In [65]:
final_urls = []
counter = 0
last_char = None
for u in urls:
    curr_char = u.split("/en/part")[1].split("/")[0]
    if last_char != curr_char:
        final_urls.append(prefix_urls[counter])
        counter += 1
        last_char = curr_char
    final_urls.append(url+u)

In [66]:
len(final_urls)

74

In [67]:
final_urls

['https://fullstackopen.com/en/part0',
 'https://fullstackopen.com/en/part0/general_info',
 'https://fullstackopen.com/en/part0/fundamentals_of_web_apps',
 'https://fullstackopen.com/en/part1',
 'https://fullstackopen.com/en/part1/introduction_to_react',
 'https://fullstackopen.com/en/part1/java_script',
 'https://fullstackopen.com/en/part1/component_state_event_handlers',
 'https://fullstackopen.com/en/part1/a_more_complex_state_debugging_react_apps',
 'https://fullstackopen.com/en/part2',
 'https://fullstackopen.com/en/part2/rendering_a_collection_modules',
 'https://fullstackopen.com/en/part2/forms',
 'https://fullstackopen.com/en/part2/getting_data_from_server',
 'https://fullstackopen.com/en/part2/altering_data_in_server',
 'https://fullstackopen.com/en/part2/adding_styles_to_react_app',
 'https://fullstackopen.com/en/part3',
 'https://fullstackopen.com/en/part3/node_js_and_express',
 'https://fullstackopen.com/en/part3/deploying_app_to_internet',
 'https://fullstackopen.com/en/pa

In [68]:
from PyPDF2 import PdfMerger, PdfReader
#pyhtml2pdf not working, code deleted. If interested you may consult with previous notebooks
#from pyhtml2pdf import converter

import weasyprint as wp
from weasyprint import HTML
import requests

def download_file(download_url, filename):
    
    # Use requests to get the HTML content of the page
    response = requests.get(download_url)
    html_content = response.content.decode('utf-8')

    # Convert the HTML content to a PDF
    HTML(string=html_content, base_url=download_url).write_pdf(filename)

In [69]:
# The above weasyprint only captures first few pages. So I try selenium 
# and copy from someone's code from Medium: https://medium.com/@nikitatonkoshkur25/create-pdf-from-webpage-in-python-1e9603d6a430
# I also tweak and fix a few things so that code can run

import base64
import json
import logging
import time
from io import BytesIO
from typing import List

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager


class PdfGenerator:
    """
     Simple use case:
        pdf_file = PdfGenerator(['https://google.com']).main()
        with open('new_pdf.pdf', "wb") as outfile:
            outfile.write(pdf_file[0].getbuffer())
    """
    driver = None
    # https://chromedevtools.github.io/devtools-protocol/tot/Page#method-printToPDF
    print_options = {
        'landscape': False,
        'displayHeaderFooter': False,
        'printBackground': True,
        'preferCSSPageSize': True,
        'paperWidth': 6.97,
        'paperHeight': 16.5,
    }

    def __init__(self, urls: List[str]):
        self.urls = urls

    def _get_pdf_from_url(self, url, *args, **kwargs):
        self.driver.get(url)

        time.sleep(0.3)  # allow the page to load, increase if needed

        print_options = self.print_options.copy()
        result = self._send_devtools(self.driver, "Page.printToPDF", print_options)
        return base64.b64decode(result['data'])

    @staticmethod
    def _send_devtools(driver, cmd, params):
        """
        Works only with chromedriver.
        Method uses cromedriver's api to pass various commands to it.
        """
        return driver.execute_cdp_cmd(cmd, params)

    def _generate_pdfs(self):
        pdf_files = []

        for url in self.urls:
            result = self._get_pdf_from_url(url)
            file = BytesIO()
            file.write(result)
            pdf_files.append(file)

        return pdf_files

    def main(self) -> List[BytesIO]:
        webdriver_options = ChromeOptions()
        webdriver_options.add_argument('--headless')
        webdriver_options.add_argument('--disable-gpu')

        try:
            self.driver = webdriver.Chrome(
                service=ChromeService(ChromeDriverManager().install()),
                options=webdriver_options
            )
            result = self._generate_pdfs()
        finally:
            self.driver.close()

        return result

In [72]:
mergedObject = PdfMerger()
    
for fileNumber in range(len(final_urls)):
    pdf_path = final_urls[fileNumber]
    file_str = str(fileNumber)+ '.pdf'

    pdf_file = PdfGenerator([pdf_path]).main()
    # save pdf to file
    with open(file_str, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
                
    mergedObject.append(PdfReader(file_str, 'rb'))

mergedObject.write("mergedfilesoutput.pdf")